In [2]:
!pip3 install pennprov

    100% |████████████████████████████████| 112kB 3.6MB/s ta 0:00:01
  Using cached https://files.pythonhosted.org/packages/d4/70/d60450c3dd48ef87586924207ae8907090de0b306af2bce5d134d78615cb/python_dateutil-2.8.1-py2.py3-none-any.whl
    100% |████████████████████████████████| 9.5MB 46kB/s  eta 0:00:01    40% |████████████▉                   | 3.8MB 5.9MB/s eta 0:00:01    77% |████████████████████████▋       | 7.3MB 10.0MB/s eta 0:00:01
    100% |████████████████████████████████| 153kB 4.6MB/s eta 0:00:01
    100% |████████████████████████████████| 143kB 3.0MB/s eta 0:00:01
    100% |████████████████████████████████| 512kB 974kB/s eta 0:00:01
    100% |████████████████████████████████| 13.4MB 58kB/s eta 0:00:011   59% |███████████████████             | 7.9MB 12.7MB/s eta 0:00:01


In [ ]:
from pennprov.connection.mprov_connection import MProvConnection
from pennprov.metadata.stream_metadata import BasicSchema, BasicTuple
from datetime import datetime, timezone

def area_circle(input):
    return {'key': input['key'],
        'value': input['value'] * input['value'] * 3.1415}

conn = MProvConnection('YOUR_USERNAME', 'YOUR_PASSWORD', 'host.docker.internal:8088')
conn.create_or_reset_graph()

# Create a node representing the collection of items in the stream, and annotate that
collection = conn.create_collection('stream1', 1, None)
conn.store_annotations(collection, {'version': 1, 'privacy': 'none'})

# Create a simple relation or stream, with a binary schema
data_schema = BasicSchema('SampleStream', {'key': 'int', 'value':'int'})


In [2]:
# Create a sample tuple
for i in range(0,10):
    tuple = BasicTuple(data_schema, {'key': i, 'value': 456 + i})
    tuple_index = i

    # Store the initial data, get the ID (token) of its node in the graph
    input_token = conn.store_stream_tuple('SampleStream', tuple_index, tuple)
    conn.add_to_collection(input_token, collection)

    # Compute an operation over the tuple, convert it to a tuple
    ts = datetime.now(timezone.utc)
    result = area_circle(tuple)
    out_tuple = BasicTuple(data_schema, result)

    # Store the derived tuple and the derivation name / time
    res = conn.store_derived_result('OutStream', tuple_index, out_tuple, input_token, 'area_circle', ts, ts)
    
    # We can annotate each individual output tuple, if we like
    conn.store_annotations(res, {'protected': True})


ApiException: (500)
Reason: Server Error
HTTP response headers: HTTPHeaderDict({'Date': 'Mon, 07 Jun 2021 19:55:14 GMT', 'Content-Type': 'text/html;charset=utf-8', 'Transfer-Encoding': 'chunked', 'Server': 'Jetty(9.4.31.v20200723)'})
HTTP response body: {"message":"SQL [insert into \"Hab_Edge\" (\"_from\", \"label\", \"_to\", \"_resource\") select ?, ?, ?, ? where not exists (select * from \"Hab_Edge\" where ((_from \u003d ?) and (_to \u003d ?) and (label \u003d ?) and (_resource \u003d ?)))]; ERROR: insert or update on table \"Hab_Edge\" violates foreign key constraint \"Hab_Edge__to_fkey\"\n  Detail: Key (_to, _resource)\u003d({http://mprov.md2k.org}u_YOUR_USERNAME2, mProv-graph) is not present in table \"Hab_Node\"."}


See what you have by clicking on http://localhost:7474 and entering `MATCH (n) RETURN n`